In [1]:
from pynq import Overlay
overlay = Overlay('/home/xilinx/pi-radio-overlays/cfo_correction/cfo_correction.bit')
overlay?

In [2]:
from pynq.lib import AxiGPIO
from pynq import allocate

# Data DMA
data_dma = overlay.axi_dma_0
# AXI GPIO
bypass_start = overlay.bypass_start
playback_symbols = overlay.playback_symbols

In [3]:
# Ensure waveform playback is inactive
bypass_start.write(0x8,0x1)  

In [4]:
# Ensure CFO is enabled
bypass_start.write(0x0,0x0)

In [5]:
# Set number of symbols to playback
playback_symbols.write(0x0,0x1)

In [6]:
# Enable playback
playback_symbols.write(0x8,0x1)

In [28]:
# Read waveform from FIFO via DMA
import numpy as np
from numpy import loadtxt

nfft = 1024
cp_len = 256
ofdm_symbols = 10
symbol_length = nfft + cp_len;
out_data_buffer = allocate(shape=((symbol_length-cp_len)*ofdm_symbols,),dtype=np.int16)

data_dma.recvchannel.transfer(out_data_buffer)
data_dma.recvchannel.wait()

out_samples = np.empty(shape=((symbol_length-cp_len)*ofdm_symbols,),dtype=np.int16)
np.copyto(out_samples,out_data_buffer)

out_data_buffer.close()

In [ ]:
# Write to text file
np.savetxt('/home/xilinx/pi-radio-overlays/cfo_correction/cfo_output_samples.txt',out_samples,fmt='%i')